# Twitter API
---

In [30]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time
from datetime import timedelta,timezone
from time import sleep
from pandas import json_normalize
from dateutil.relativedelta import relativedelta
from typing import Dict, List
import logging
from pandas import DataFrame
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from dateutil import parser
from datetime import datetime
import pytz

#### Twitter API KEY & Token for the developer account of vithu92

In [ ]:
# add your bear token here
BEAR_TOKEN = ""
os.environ['TWITTER_TOKEN'] = BEAR_TOKEN

#### Functions implemented for fetching data from Twitter API

In [ ]:
class TwitterAPI:
    request_counter:int = 0
    request_limit:int = 300
    def __init__(self):
        self.request_counter = 0
    def auth(self):
        return os.getenv('TWITTER_TOKEN')

    def create_headers(self,bearer_token):
        headers = {"Authorization": "Bearer {}".format(bearer_token)}
        return headers

    def create_csv_from_json(self,filename:str, keyword:str):
        with open(f'results/json/{filename}.json') as json_file:
            data = json.load(json_file)
            df = json_normalize(data['data'])
        with open(f"results/json/{filename}_user.json") as json_file:
            user = json.load(json_file)
            df_user = json_normalize(user['users'])
            df_user.rename(columns={'id':'author_id'}, inplace=True)
        df_merge = pd.merge(df, df_user, on='author_id')
        df_merge.insert(0, 'Keyword', f'{keyword}')
        df_merge.to_csv(f"results/csv/{filename}.csv")

    def get_tweeter_user(self,user_id:str=None, username:str=None):
        headers = self.create_headers(self.auth())
        if user_id:
            search_url = f"https://api.twitter.com/2/users/{user_id}"
        if username:
            search_url = f"https://api.twitter.com/2/users/by/username/{username}"

        query_params = {'user.fields': 'public_metrics'}
        response = self.connect_to_endpoint(url=search_url, headers=headers, params=query_params, next_token=None)
        return response['data']


    def create_url(self,keyword:str,griffin:bool,start_date:str, end_date:str,years:int=0,max_results = 500):

        search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

        #change params based on the endpoint you are using
        #start_date = datetime.datetime.now(datetime.timezone.utc) - timedelta(days=365*years)
        #start_date = start_date.isoformat()
        #end_date = datetime.datetime.now(datetime.timezone.utc) - timedelta(days=365*(years-1))
        #end_date = end_date.isoformat()
        print(start_date)
        print(end_date)
        if griffin:
            start_date = datetime.strptime(start_date, '%d.%m.%Y')
            start_date = start_date.astimezone(pytz.UTC)
            end_date = datetime.strptime(end_date, '%d.%m.%Y')
            end_date = end_date.astimezone(pytz.UTC)
            query_params = {'query': keyword,
                           'start_time': start_date.isoformat(),
                            'end_time': end_date.isoformat(),
                            'tweet.fields':'author_id,created_at,in_reply_to_user_id,public_metrics,entities',
                            'expansions': 'attachments.media_keys,author_id,in_reply_to_user_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id',
                            'user.fields': 'id,name,public_metrics,username',
                            'media.fields':'url',
                            'max_results': max_results,
                            'sort_order':"recency",
                            'next_token':{}}
        else:
            query_params = {'query': keyword,
                            'start_time': start_date,
                            #'end_time': end_date,
                            'max_results': max_results,
                            'expansions': 'attachments.poll_ids,attachments.media_keys,author_id,geo.place_id,in_reply_to_user_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id',
                            'tweet.fields': "attachments,created_at,author_id",
                            'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
                            'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
                            'media.fields':'duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width',
                            'next_token': {}}
        return (search_url, query_params)


    def connect_to_endpoint(self,url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        retry_strategy = Retry(
            total=3,
            status_forcelist=[429, 500, 502, 503, 504],
            method_whitelist=["HEAD", "GET", "OPTIONS"]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("https://", adapter)
        http.mount("http://", adapter)
        
        response = http.get(url=url, headers = headers, params = params)
        
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)        
        sleep(2)
        return response.json()

    def get_date_range(self,delta:int=7):
        end_date = datetime.datetime.now(datetime.timezone.utc)
        end_date = end_date - timedelta(hours=1)
        start_date = end_date - timedelta(days=delta)
        start_date = start_date + timedelta(hours=1)
        return start_date.isoformat(), end_date.isoformat()

    def cast_dict(self,x, name):
        d = {}
        if isinstance(x,list):
            for k,v in x[0].items():
                 d["{}_{}".format(name, k)] = v
        return pd.Series(d)
    
    def get_user_dataframe(self,data:dict):
        row_list = []
    
        for user in data['includes']['users']:
            user_dict={}
            user_dict['favourites_count'] = 0
            user_dict['followers_count'] = user.get("public_metrics",{}).get('followers_count',"")
            user_dict['friends_count'] =  user.get("public_metrics",{}).get('following_count',"")
            user_dict['listed_count']= user.get("public_metrics",{}).get('listed_count',"")
            user_dict['statuses_count'] = user.get("public_metrics",{}).get('tweet_count',"")
            user_dict['name'] = user.get('name',"")
            user_dict['screen_name'] = user.get('username',"")
            user_dict['id_str'] =  user.get('id',"")

            row_list.append(user_dict)
        df = pd.DataFrame(row_list)
        df_media_outlets = pd.read_csv("media_outlets.csv")
        media_list = df_media_outlets['@name'].apply(lambda x: x[1:]).tolist()
        for index, row in df.iterrows():
            if row['screen_name'] in media_list:
                df.at[index,'media_outlet'] = True
            else:
                df.at[index, 'media_outlet'] = False
        
        return df

    def get_media_dataframe(self, data:dict):
        row_list =[]
        if len(data.get('includes','').get('media','')) == 0:
            return None
        for media in data['includes']['media']:
            media_dict = {}
            #['media_key', 'type', 'url']
            media_dict['media_key'] = media.get('media_key','')
            media_dict['type'] = media.get('type','')
            media_dict['url'] = media.get('url','')
            row_list.append(media_dict)
        
        return pd.DataFrame(row_list)


    def get_tweet_dataframe(self,data:dict):
        row_list = []
        for entry in data['data']:
            data_dict = {}
            data_dict['text'] = entry.get('text',"")
            data_dict['created_at'] = entry.get('created_at',"")
            data_dict['id_str'] = entry.get('author_id',"")
            # hashtags
            hashtags_list = entry.get('entities',{}).get("hashtags",[])
            if len(hashtags_list) != 0:
                data_dict['hashtags'] = hashtags_list[0].get('tag','')

            else:
                data_dict['hashtags'] = ""
            # destination user
            mentions_list = entry.get('entities',{}).get("mentions",[])
            if len(mentions_list) != 0:
                data_dict['dst_id_str'] = mentions_list[0].get('id',"")
            else:
                data_dict['dst_id_str'] = ""
            media_list = entry.get('attachments',{}).get('media_keys',[])
            if len(media_list) != 0:
                data_dict['media_keys'] = media_list
            else:
                data_dict['media_keys'] = ""
            row_list.append(data_dict)
            
        return pd.DataFrame(row_list)

    def add_prefix_to_user_columns(self,df:DataFrame, prefix:str):
        column_list = ['favourites_count', 'followers_count', 'friends_count', 'listed_count',
           'statuses_count', 'name', 'screen_name', 'id_str']
        new_names = [(i,f"{prefix}_"+i) for i in column_list]
        df_renamed = df.rename(columns = dict(new_names))

        return df_renamed

    def get_data_dataframe(self,data:dict):
        df_user = self.get_user_dataframe(data=data)
        df_tweet= self.get_tweet_dataframe(data=data)
        # merging src user
        df_total = pd.merge(left=df_tweet, right=df_user, how="left", on=['id_str'])
        df_total = self.add_prefix_to_user_columns(df_total, prefix="src")
        # mergin dst 
        df_total = df_total.rename(columns={"dst_id_str": "id_str"})
        df_user_dest = df_user[['name', 'screen_name', 'id_str']]
        df_total = pd.merge(left=df_total, right=df_user_dest, how="left", on=['id_str'])
        df_total = self.add_prefix_to_user_columns(df_total, prefix="dst")
    
        # adding custom columns
        df_total['src'] = df_total['src_screen_name'].str.lower()
        df_total['dst'] = df_total['dst_screen_name'].str.lower()
        
        return df_total, df_user

    def get_tweets_griffin(self,keyword:str, start_date:str, end_date:str,years:int=0,media:bool=False,limit:int=None):
        headers = self.create_headers(self.auth())
        search_url, params = self.create_url(keyword=keyword, years=years, griffin=True,start_date=start_date, end_date=end_date,max_results=500)
        json_response = self.connect_to_endpoint(url=search_url, headers=headers, params=params)
        df,df_user = self.get_data_dataframe(data=json_response)
        if media:
            df_media = self.get_media_dataframe(data=json_response)
        tweets_counter = json_response['meta']['result_count']
        print(f"Fetched {tweets_counter} tweets",end='\r')
        while 'next_token' in json_response['meta']:
            try:
                json_response = self.connect_to_endpoint(url=search_url, headers=headers, params=params, next_token=str(json_response['meta']['next_token']))
            except:
                break
            
            if isinstance(df, pd.DataFrame):
                df_tmp, df_user_tmp = self.get_data_dataframe(data=json_response)
                df = df.append(df_tmp)
                df_user = df_user.append(df_user_tmp)
            
            if media:
                if isinstance(df_media, pd.DataFrame):
                    if len(json_response.get('includes','').get('media','')) != 0:
                        df_media = df_media.append(self.get_media_dataframe(data=json_response))
            tweets_counter += json_response['meta']['result_count']
            print(f"Fetched {tweets_counter} tweets",end='\r')
            if limit:
                if tweets_counter >= limit:
                    break
            
        
        # add keyword as column
        df['twitter_query'] = keyword
        
        # timestamp
        df['time'] = df['created_at'].apply(lambda x: (int((parser.parse(x)).timestamp())))
        
        #  rearranging
        #df = df.reindex(columns=['src_favourites_count', 'src_followers_count', 'src_friends_count',
       #'src_listed_count', 'src_statuses_count', 'src_name', 'src_screen_name',
       #'src_id_str', 'text', 'created_at', 'twitter_query', 'hashtags',
       #'dst_name', 'dst_screen_name', 'dst_id_str', 'time', 'src', 'dst'])
        
        # cleaning
        df = self.clean_edges(df)
        df_user = self.clean_nodes(df_user)
        
        if media:
            for index, row in df.iterrows():
                if row['media_keys']:
                    for entry in row['media_keys']:
                        index = df_media.index[df_media['media_key'].str.contains(entry)].tolist()[0]
                        df_media.at[index,'src'] = row['src']
                        df_media.at[index,'src_id_str'] = row['src_id_str']
            
            df = df.drop("media_keys",axis=1)
            return df, df_user, df_media
        else:
            return df, df_user
    
    def clean_edges(self, df):
        #clean twitter text form unwanted characters
        df['text'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
        df['text'] = df['text'].replace({'"': '' }, regex=True)
        df['text'] = df['text'].replace({';': ',' }, regex=True)
        df['text'] = df['text'].replace({' &amp;': '' }, regex=True)
        df['text'] = df['text'].replace({'&amp;': '' }, regex=True)
        #clean source- and destination-name from unwanted characters
        df['src_name'] = df['src_name'].replace({'\'': '' }, regex=True)
        df['dst_name'] = df['dst_name'].replace({'\'': '' }, regex=True)
        df['src_name'] = df['src_name'].replace({';': ',' }, regex=True)
        df['dst_name'] = df['dst_name'].replace({';': ',' }, regex=True)
        #format the datetime to be griffin readable
        df['created_at'] = pd.to_datetime(df["created_at"], format="%Y-%m-%dT%H:%M:%S.%f")
        return df

    def clean_nodes(self, df):
        #clean name from unwanted characters
        df['name'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
        df['name'] = df['name'].replace({'"': '' }, regex=True)
        df['name'] = df['name'].replace({'\'': '' }, regex=True)
        df['name'] = df['name'].replace({' &amp;': '' }, regex=True)
        df['name'] = df['name'].replace({';': ',' }, regex=True)
        return df

    def get_tweets(self,keyword:str, filename:str, years:int = 10,griffin:bool=False,limit:int=None):
        headers = self.create_headers(auth())
        start, end = self.get_date_range()
        search_url, params = self.create_url(keyword=keyword, years=years, griffin=griffin)
        df_twitter = pd.DataFrame()


        json_response = self.connect_to_endpoint(url=search_url, headers=headers, params=params)
        counter = 0
        tweets_counter = 0
        tweets_counter += json_response['meta']['result_count']
        length_df = 0
        if not 'data' in json_response:
            print(f"No results found for keyword: {keyword}")
            return
        try:
            while json_response['meta']['next_token']:
                #search_url, params = create_url(keyword=keyword,years=years)
                json_response = self.connect_to_endpoint(url=search_url, headers=headers, params=params, next_token=str(json_response['meta']['next_token']))

                #json_file_data.writelines(
                #    "," + json.dumps(json_response["data"])[1:-1])
                #json_file_user.writelines("," + json.dumps(json_response['includes']['users'])[1:-1])

                #normalize tweets and change type to datetime
                tweets = pd.json_normalize(json_response['data'])
                length_df += len(tweets)
                print('number of new rows: ' +  str(len(tweets)))
                print('expected number of rows in total: ' + str(length_df))
                tweets['created_at'] = pd.to_datetime(tweets['created_at']).dt.strftime('%d.%m.%Y %H:%M:%S')

                #cast nested dictionaries to seperate columns with a prefix
                tweets = tweets.join(tweets['referenced_tweets'].apply(cast_dict, name = 'referenced_tweets'), how = 'outer')\
                        .drop("referenced_tweets", axis=1)
                tweets = tweets.join(tweets['entities.mentions'].apply(cast_dict, name = 'entities.mentions'), how = 'outer')\
                        .drop("entities.mentions", axis=1)
                #cast entry in list to column (as it is needed as a key to join media)
                tweets['attachments.media_keys'] = tweets['attachments.media_keys'].apply(lambda x: x[0] if isinstance(x,list) else None)
                #add a prefix tweet
                tweets = tweets.add_prefix('tweet.')

                #normalize media and add a prefix to columnnames
                media = pd.json_normalize(json_response['includes']['media'])
                media = media.add_prefix('media.')

                #normalize users, change type to datetime and add a prefix to columnnames
                users = pd.json_normalize(json_response['includes']['users'])
                users['created_at'] = pd.to_datetime(users['created_at']).dt.strftime('%d.%m.%Y %H:%M:%S')
                users = users.add_prefix('user.')

                #merge both media and users with the tweets to get all the information in one dataframe
                df_total = pd.merge(tweets, media, left_on = 'tweet.attachments.media_keys', right_on = 'media.media_key', how="left")
                df_total = pd.merge(df_total, users, left_on = 'tweet.author_id', right_on = 'user.id', how="left")
                df_twitter = df_twitter.append(df_total, ignore_index = True)
                print('actual number of rows in total: ' + str(len(df_twitter)))
                print(tweets_counter)

                counter += 1
                tweets_counter += json_response["meta"]["result_count"]
                if limit:
                    if tweets_counter > limit:
                        break
        except:
            print("no next token")
            df_twitter.to_json(f"results/json/{filename}.json")
            df_twitter.to_csv(f"results/csv/{filename}.csv")
            return
    
    #create_csv_from_json(filename=filename, keyword=keyword)
    #df_twitter.to_json(f"results/json/{keyword}.json")

### Main

In [ ]:
twitter_api = TwitterAPI()

# the keyword is the search term in twitter api - 
# explaination for this search team: look for all tweets in english with the substring/hashtag sunrisemvmt
# start_date = setup the start date - i did it because otherwise it took my more than 24hourse to fetch all data
# end_date => same the format is dd.mm.yyyy

# media = Boolean to tell if you need a seperate csv with all the media files (url, user_id)

#output: 2 datafraeme:
# 1. dataframe = df (tweeter data) => this is going to be the edges
# 2. dataframe = user data => this is going to be the nodes
df,df_user, df_media = twitter_api.get_tweets_griffin(keyword="lang:en (sunrisemvmt)", start_date="17.05.2021",end_date="16.05.2022", media=True)

In [62]:
# export as csv
df_user.to_csv("hashtags_sunrisemvmt_nodes.csv", index=False)
df.to_csv("hashtags_sunrisemvmt_edges.csv", index=False)
df_media.to_csv("hashtags_sunrisemvmt_media.csv", index=False)